In [1]:
import tensorflow as tf

2025-07-26 16:31:20.875832: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-26 16:31:20.903478: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-26 16:31:20.903522: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-26 16:31:20.922103: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-26 16:31:21.856188: W tensorflow/com

In [17]:
import boto3
import tensorflow as tf
import tarfile
import os
import sagemaker
import shutil
import numpy as np

# ==============================================================================
# ETAPA 1: PREPARAÇÃO E EMPACOTAMENTO
# ==============================================================================

print("--- INICIANDO ETAPA 1: PREPARAÇÃO E EMPACOTAMENTO ---")

# --- 1. CONFIGURAÇÃO INICIAL ---
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
s3_client = boto3.client('s3')

# --- CONFIGURE OS CAMINHOS DO SEU BUCKET S3 AQUI ---
# Substitua com o nome do seu bucket e os caminhos corretos
s3_bucket_name = 'sagemaker-us-east-2-982534382598'

# Caminhos dos artefatos de ENTRADA no S3
input_model_s3_key = 'lstm-stock-prediction-artifacts/lstm_stock_prediction_model.h5'      # SUBSTITUA
input_scaler_s3_key = 'lstm-stock-prediction-artifacts/scaler.joblib' # SUBSTITUA

# Caminho de SAÍDA no S3 onde o artefato final será salvo
output_s3_key_prefix = 'sagemaker-lstm-artifacts/previsao-acoes'

# Nomes dos arquivos locais temporários
local_h5_path = 'modelo_local.h5'
local_scaler_path = 'scaler_local.joblib'

# Estrutura de diretórios para empacotamento
export_dir = 'export'
saved_model_path = os.path.join(export_dir, 'Servo/1')
code_dir = os.path.join(export_dir, 'code')

# Limpa execuções anteriores para garantir um ambiente limpo
if os.path.exists(export_dir): shutil.rmtree(export_dir)
if os.path.exists('model.tar.gz'): os.remove('model.tar.gz')
os.makedirs(saved_model_path, exist_ok=True)
os.makedirs(code_dir, exist_ok=True)


# --- 2. BAIXAR MODELO E SCALER DO S3 ---
print(f"Baixando modelo de s3://{s3_bucket_name}/{input_model_s3_key}...")
s3_client.download_file(s3_bucket_name, input_model_s3_key, local_h5_path)

print(f"Baixando scaler de s3://{s3_bucket_name}/{input_scaler_s3_key}...")
s3_client.download_file(s3_bucket_name, input_scaler_s3_key, local_scaler_path)
print("Downloads concluídos.")


# --- 3. CRIAR 'inference.py' E 'requirements.txt' ---
# Criando o requirements.txt para instalar dependências
requirements_path = os.path.join(code_dir, 'requirements.txt')
with open(requirements_path, 'w') as f:
    f.write('scikit-learn\njoblib')

# Conteúdo do inference.py para o modelo LSTM
inference_code = """
import json
import numpy as np
import os
import joblib

scaler = None

def _load_scaler():
    global scaler
    if scaler is None:
        print("Scaler não está em memória. Carregando de 'scaler.joblib'...")
        scaler_path = os.path.join('/opt/ml/model/code', 'scaler_local.joblib')
        if os.path.exists(scaler_path):
            scaler = joblib.load(scaler_path)
            print("Scaler carregado com sucesso.")
        else:
            raise FileNotFoundError(f"ERRO CRÍTICO: 'scaler.joblib' não encontrado em {scaler_path}")

def input_handler(data, context):
    print("Executando input_handler...")
    _load_scaler()

    input_data = json.loads(data.read().decode('utf-8'))['instances']
    # input_array terá o shape (1, 60)
    input_array = np.array(input_data, dtype=np.float32)
    input_reshaped_for_scaler = input_array.reshape(-1, 1)

    # Agora o scaler recebe os dados no formato esperado (60 amostras, 1 característica).
    scaled_input = scaler.transform(input_reshaped_for_scaler) # O resultado terá o shape (60, 1)

    # Remodelar os dados normalizados para o formato que o modelo LSTM espera: (1, 60, 1)
    # Formato: [batch_size, timesteps, features]
    reshaped_for_model = scaled_input.reshape(1, -1, 1)

    return json.dumps({"instances": reshaped_for_model.tolist()})

def output_handler(response, context):
    print("Executando output_handler...")
    _load_scaler()

    if response.status_code != 200:
        raise ValueError(f"Erro do servidor de modelos: {response.content.decode('utf-8')}")

    tfs_response = json.loads(response.content.decode('utf-8'))
    
    if 'predictions' not in tfs_response:
        raise ValueError(f"Resposta do modelo não contém a chave 'predictions': {tfs_response}")
        
    scaled_prediction = np.array(tfs_response['predictions'])

    # A predição do modelo (shape 1,1) já está no formato correto para inverse_transform.
    real_prediction = scaler.inverse_transform(scaled_prediction)

    result = {"predicted_price": real_prediction.tolist()[0][0]}
    
    # --- CORREÇÃO APLICADA AQUI ---
    # Retornando uma tupla com a resposta e o content-type, como o serviço espera.
    return json.dumps(result), context.accept_header
"""
inference_script_path = os.path.join(code_dir, 'inference.py')
with open(inference_script_path, 'w') as f:
    f.write(inference_code)
print("Scripts de inferência e dependências criados.")


# --- 4. CONVERTER MODELO E ORGANIZAR ARTEFATOS ---
print(f"Convertendo '{local_h5_path}' para o formato SavedModel...")
model = tf.keras.models.load_model(local_h5_path)
tf.saved_model.save(model, saved_model_path)

# Copiar o scaler baixado para o diretório de código
shutil.copy(local_scaler_path, code_dir)
print("Modelo convertido e scaler posicionado.")


# --- 5. EMPACOTAR E FAZER UPLOAD ---
output_tar_path = 'model.tar.gz'
print(f"Criando o arquivo '{output_tar_path}'...")
with tarfile.open(output_tar_path, 'w:gz') as tar:
    tar.add(export_dir, arcname='.')

output_model_path_s3 = f"s3://{s3_bucket_name}/{output_s3_key_prefix}/model.tar.gz"
print(f"Fazendo upload para {output_model_path_s3}...")
sagemaker_session.upload_data(path=output_tar_path, key_prefix=output_s3_key_prefix)
print("--- ETAPA 1 CONCLUÍDA COM SUCESSO ---")


# ==============================================================================
# ETAPA 2: DEPLOY DO ENDPOINT
# ==============================================================================

print("\n--- INICIANDO ETAPA 2: DEPLOY DO ENDPOINT ---")

# --- 1. CRIAR O OBJETO DO MODELO ---
sagemaker_model = TensorFlowModel(
    model_data=output_model_path_s3,
    role=role,
    framework_version='2.12', # Use a mesma versão principal do TF usada no treinamento
    sagemaker_session=sagemaker_session,
    entry_point='inference.py'
)

# --- 2. IMPLANTAR O MODELO ---
print("Iniciando o deploy do endpoint. Isso pode levar vários minutos...")
predictor = sagemaker_model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium'
)
print(f"\n✅ Deploy concluído com sucesso!")
print(f"Nome do Endpoint: {predictor.endpoint_name}")




--- INICIANDO ETAPA 1: PREPARAÇÃO E EMPACOTAMENTO ---
Baixando modelo de s3://sagemaker-us-east-2-982534382598/lstm-stock-prediction-artifacts/lstm_stock_prediction_model.h5...
Baixando scaler de s3://sagemaker-us-east-2-982534382598/lstm-stock-prediction-artifacts/scaler.joblib...
Downloads concluídos.
Scripts de inferência e dependências criados.
Convertendo 'modelo_local.h5' para o formato SavedModel...
INFO:tensorflow:Assets written to: export/Servo/1/assets


INFO:tensorflow:Assets written to: export/Servo/1/assets


Modelo convertido e scaler posicionado.
Criando o arquivo 'model.tar.gz'...
Fazendo upload para s3://sagemaker-us-east-2-982534382598/sagemaker-lstm-artifacts/previsao-acoes/model.tar.gz...
--- ETAPA 1 CONCLUÍDA COM SUCESSO ---

--- INICIANDO ETAPA 2: DEPLOY DO ENDPOINT ---
Iniciando o deploy do endpoint. Isso pode levar vários minutos...
-----!
✅ Deploy concluído com sucesso!
Nome do Endpoint: tensorflow-inference-2025-07-26-19-17-42-056


In [29]:
import yfinance as yf
ACAO = 'ITUB'            # Símbolo da empresa (ex: 'DIS' para Disney)
START_DATE = '2025-05-25'  # Data de início da coleta de dados
END_DATE = '2025-07-24'    # Data de fim da coleta de dados
SEQUENCE = 60       # Número de dias passados para prever o próximo dia (timestep do LSTM)

df = yf.download(ACAO, start=START_DATE, end=END_DATE)

test_60 = [ i[0] for i in df['Close'].values.tolist()]

/tmp/ipykernel_26911/682262794.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ACAO, start=START_DATE, end=END_DATE)
[*********************100%***********************]  1 of 1 completed


In [30]:
# --- 3. TESTAR O ENDPOINT ---
print("\nEnviando dados de teste para o endpoint...")
sequence_length = 60 # Ajuste para o tamanho da sequência do seu modelo
test_sequence = np.random.uniform(low=100.0, high=150.0, size=(1, sequence_length)).tolist()

payload = {"instances": test_60}

try:
    response = predictor.predict(payload)
    print("\nPredição recebida do endpoint:")
    print(response)
except Exception as e:
    print(f"\n❌ Ocorreu um erro ao invocar o endpoint: {e}")

# --- 4. LIMPEZA (OPCIONAL) ---
# print("\nPara deletar o endpoint, remova o comentário da linha abaixo e execute novamente:")
# predictor.delete_endpoint()


Enviando dados de teste para o endpoint...

Predição recebida do endpoint:
{'predicted_price': 6.31754140722038}


In [31]:
test_60

[6.730000019073486,
 6.639999866485596,
 6.630000114440918,
 6.590000152587891,
 6.539999961853027,
 6.590000152587891,
 6.53000020980835,
 6.519999980926514,
 6.559999942779541,
 6.559999942779541,
 6.539999961853027,
 6.579999923706055,
 6.610000133514404,
 6.559999942779541,
 6.739999771118164,
 6.739999771118164,
 6.679999828338623,
 6.630000114440918,
 6.650000095367432,
 6.789999961853027,
 6.590000152587891,
 6.599999904632568,
 6.619999885559082,
 6.789999961853027,
 6.789999961853027,
 6.809999942779541,
 6.920000076293945,
 6.840000152587891,
 6.809999942779541,
 6.650000095367432,
 6.369999885559082,
 6.300000190734863,
 6.269999980926514,
 6.289999961853027,
 6.329999923706055,
 6.449999809265137,
 6.300000190734863,
 6.389999866485596,
 6.309999942779541,
 6.449999809265137]